<a href="https://colab.research.google.com/github/anjunjin/PFE-ING3-IA/blob/master/Nettoyages_des_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

On peut Définir la seed pour que l'aléatoire soit consistant entre exécution

In [8]:
seed=42

Les pip install

# Import de package et connexion à Google Drive

Il faut lancer ce bloc de code 2 fois

In [9]:
%pip install --upgrade pandas

In [10]:
pip install emot --upgrade

In [11]:
import pickle
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')
DATA_PATH = "/content/drive/My Drive/PFE/"
"""with open(DATA_PATH+"data_tweets.txt", "rb") as file :  
    data_tweets = pickle.load(file)"""
data_tweets = pd.read_pickle(DATA_PATH+"data_tweets.pkl")

Mounted at /content/drive


In [12]:
import copy

from tqdm.notebook import tqdm
tqdm.pandas()

926188 tweets sur txt, 998023 tweets sur pkl

In [13]:
data_tweets

,user_id,user_screen_name,id_tweet,created_at,tweet,retweet_count,favorite_count,liability,liability_label,count_followers
0,78957336,france_soir,1507008907390062602,2022-03-24 14:58:17,"🎙️ ""Un Corse indépendantiste, c’est à la base ...",3,11,2,Site non fiable,126112
1,78957336,france_soir,1506984886061699072,2022-03-24 13:22:49,🍄 La start-up Mycophyto propose d’enrichir le...,8,31,2,Site non fiable,126112
2,78957336,france_soir,1506959719319093249,2022-03-24 11:42:49,📽 Dans un debriefing exclusif conduit avec Ari...,54,85,2,Site non fiable,126112
3,78957336,france_soir,1506958411115675650,2022-03-24 11:37:37,⚠️ Le conflit #UkraineRussie créant à la fois ...,21,36,2,Site non fiable,126112
4,78957336,france_soir,1506953912259121156,2022-03-24 11:19:45,"🖍️ Le dessin du jour sur #McKinseyGate, par Z_...",213,350,2,Site non fiable,126112
...,...,...,...,...,...,...,...,...,...,...
997975,14354304,wikiHow,925845110461534209,2017-11-01 22:00:50,A little bit of gentle heat from a hair dryer ...,0,4,4,Site réputé fiable,73816
997976,14354304,wikiHow,925843158973366272,2017-11-01 21:53:05,@pxkelley @hiringourheroes Go out there and ge...,0,0,4,Site réputé fiable,73816
997977,14354304,wikiHow,925814991772676096,2017-11-01 20:01:09,#wikitips https://t.co/58AmUzhsaj https://t.co...,0,2,4,Site réputé fiable,73816
997978,14354304,wikiHow,925784921842634753,2017-11-01 18:01:40,If your dishwasher still holds water when you ...,1,3,4,Site réputé fiable,73816


# Préparation des éléments pour le nettoyage

In [14]:
import emot
import re
import numpy as np

In [15]:
!python -m spacy download fr_core_news_md

     |████████████████████████████████| 88.6 MB 1.3 MB/s 
  Created wheel for fr-core-news-md: filename=fr_core_news_md-2.2.5-py3-none-any.whl size=90338488 sha256=100556b5aeaba3c8ff8b1c3cafcbf27671cf54921943c0ba64cad473ebeb44ce
  Stored in directory: /tmp/pip-ephem-wheel-cache-af7piv3x/wheels/2e/26/ff/ce93eb966e7176ebe81e6c98209582e13e108cdd2d6d636df0
Successfully built fr-core-news-md
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_md')


In [16]:
import spacy
import fr_core_news_md

In [17]:
nlp = fr_core_news_md.load()

In [18]:
df_group = data_tweets.copy()
df_group = df_group.drop(columns=['user_id',
                                  'id_tweet',
                                  'created_at'
                                  ]
                         ).reset_index(drop=True)

df_group = df_group[['liability', 
                     'liability_label',
                     'user_screen_name',
                     'retweet_count',
                     'favorite_count',
                     'count_followers', 
                     'tweet'
                     ]]

On retire les tweets parodiques.
Pour les autres tweets on remplace les valeurs liability et liability_label pour simplifier.

In [19]:
df_group = df_group.drop(df_group[df_group.liability == 1].index)
df_group.loc[df_group['liability'] == 2, 'liability_label'] = 'site plutôt douteux'
df_group.loc[df_group['liability'] == 3, 'liability_label'] = 'site plutôt douteux'
df_group.loc[df_group['liability'] == 4, 'liability_label'] = 'site plutôt fiable'
df_group.loc[df_group['liability'] == 2, 'liability'] = 1
df_group.loc[df_group['liability'] == 3, 'liability'] = 1
df_group.loc[df_group['liability'] == 4, 'liability'] = 0
df_group = df_group.reset_index(drop=True)

In [20]:
df_group

,liability,liability_label,user_screen_name,retweet_count,favorite_count,count_followers,tweet
0,1,site plutôt douteux,france_soir,3,11,126112,"🎙️ ""Un Corse indépendantiste, c’est à la base ..."
1,1,site plutôt douteux,france_soir,8,31,126112,🍄 La start-up Mycophyto propose d’enrichir le...
2,1,site plutôt douteux,france_soir,54,85,126112,📽 Dans un debriefing exclusif conduit avec Ari...
3,1,site plutôt douteux,france_soir,21,36,126112,⚠️ Le conflit #UkraineRussie créant à la fois ...
4,1,site plutôt douteux,france_soir,213,350,126112,"🖍️ Le dessin du jour sur #McKinseyGate, par Z_..."
...,...,...,...,...,...,...,...
921571,0,site plutôt fiable,wikiHow,0,4,73816,A little bit of gentle heat from a hair dryer ...
921572,0,site plutôt fiable,wikiHow,0,0,73816,@pxkelley @hiringourheroes Go out there and ge...
921573,0,site plutôt fiable,wikiHow,0,2,73816,#wikitips https://t.co/58AmUzhsaj https://t.co...
921574,0,site plutôt fiable,wikiHow,1,3,73816,If your dishwasher still holds water when you ...


In [21]:
save_dataset = copy.deepcopy(df_group)

In [22]:
save_dataset

,liability,liability_label,user_screen_name,retweet_count,favorite_count,count_followers,tweet
0,1,site plutôt douteux,france_soir,3,11,126112,"🎙️ ""Un Corse indépendantiste, c’est à la base ..."
1,1,site plutôt douteux,france_soir,8,31,126112,🍄 La start-up Mycophyto propose d’enrichir le...
2,1,site plutôt douteux,france_soir,54,85,126112,📽 Dans un debriefing exclusif conduit avec Ari...
3,1,site plutôt douteux,france_soir,21,36,126112,⚠️ Le conflit #UkraineRussie créant à la fois ...
4,1,site plutôt douteux,france_soir,213,350,126112,"🖍️ Le dessin du jour sur #McKinseyGate, par Z_..."
...,...,...,...,...,...,...,...
921571,0,site plutôt fiable,wikiHow,0,4,73816,A little bit of gentle heat from a hair dryer ...
921572,0,site plutôt fiable,wikiHow,0,0,73816,@pxkelley @hiringourheroes Go out there and ge...
921573,0,site plutôt fiable,wikiHow,0,2,73816,#wikitips https://t.co/58AmUzhsaj https://t.co...
921574,0,site plutôt fiable,wikiHow,1,3,73816,If your dishwasher still holds water when you ...


Supression des tweets anglais et italiens. 

In [23]:
! pip install stop-words

  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32911 sha256=c69dc11f89d32212aa07b4c0dcdf2613889a269eeaf03f653e09ca36e8cb93f4
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [24]:
! pip install advertools

     |████████████████████████████████| 287 kB 26.9 MB/s 
     |████████████████████████████████| 264 kB 44.7 MB/s 
     |████████████████████████████████| 47 kB 4.2 MB/s 
     |████████████████████████████████| 3.1 MB 14.4 MB/s 
     |████████████████████████████████| 55 kB 3.7 MB/s 
     |████████████████████████████████| 3.6 MB 42.0 MB/s 
     |████████████████████████████████| 87 kB 4.1 MB/s 
     |████████████████████████████████| 251 kB 11.5 MB/s 
     |████████████████████████████████| 74 kB 2.2 MB/s 
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-py3-none-any.whl size=11516 sha256=f6d809dffab985ff02e284938fc99c0718a2877e50cf3747757d0b650389da03
  Stored in directory: /root/.cache/pip/wheels/2d/18/21/3c6a732eaa69a339198e08bb63b7da2c45933a3428b29ec454
Successfully built PyDispatcher


In [25]:
#On récupère les stopwords italiens et anglais
from stop_words import get_stop_words
import advertools as adv
stop_words_en = get_stop_words('en')
stop_words_it = get_stop_words('it')
stop_words_arb = get_stop_words('arabic')
stop_words_jap = adv.stopwords['japanese']
stop_words_ch = adv.stopwords['chinese']

In [26]:
#On enlève les mots italiens trop commun avec des mots français pour éviter la confusion
stop_words_it.remove('tu')
stop_words_it.remove('un')
stop_words_it.remove('lui')
stop_words_it.remove('ma')
stop_words_it.remove('non')
stop_words_it.remove('si')
stop_words_it.remove('se')
stop_words_it.remove('qui')
stop_words_it.remove('te')
stop_words_it.remove('sui')
stop_words_it.remove('c')
stop_words_it.remove('e')
stop_words_it.remove('i')
stop_words_it.remove('l')
stop_words_it.remove('o')
stop_words_it.remove('è')
stop_words_it.remove('a')
stop_words_it.remove('le')
stop_words_it.remove('la')
stop_words_it.remove('il')

In [27]:
#Fonction pour détecter si la phrase est oui ou non italienne ou anglaise
def detect_language(tweet):
  tweet_toLow = tweet.lower()
  tweetList = tweet_toLow.split(" ")
  nbCommon_en = len(list(set(tweetList)&set(stop_words_en)))
  nbCommon_ar = len(list(set(tweetList)&set(stop_words_arb)))
  nbCommon_jp = len(list(set(tweetList)&set(stop_words_jap)))
  nbCommon_it = len(list(set(tweetList)&set(stop_words_it)))
  nbCommon_ch = len(list(set(tweetList)&set(stop_words_ch)))
  if nbCommon_en > 2 or nbCommon_it > 2 or nbCommon_ar > 2 or nbCommon_jp > 2 or nbCommon_ch > 2:
    return True
  else: 
    return False

In [28]:
#On supprime les éléments de la dataframe présentant des tweets anglais ou italiens
df_indexs= []
c = 0
for sentence in save_dataset['tweet']: 
  if detect_language(sentence):
      df_indexs.append(c)
  c = c+1
save_dataset = save_dataset.drop(labels=df_indexs, axis=0) 
save_dataset = save_dataset.reset_index(drop=True)

In [29]:
df_tweets_clean = pd.DataFrame(save_dataset)
df_tweets_clean['cleaned'] = df_tweets_clean['tweet']

On sauvegarde le nettoyage des tweets dans un autre tableau

In [30]:
# créer un objet : emot_obj 
emot_obj = emot.core.emot()

In [31]:
# fonction (convert_emojis_to_text) : convertir les emojis en texte
# Entrée : text(un tweet)
# Sortie : text(après convertir)
def convert_emojis_to_text(text):
  try:
    # créer un objet : emot_obj 
    # emot_obj = emot.core.emot()
    # les informations sont dans ce objet.emoji 
    emoji = emot_obj.emoji(text)
    # print('List emoji :     ', emoji) 
  
    # print('Text original :                 ', text)

    for i in range(len(emoji['value'])):
      text = re.sub(emoji['value'][i], " desident_emot_"+emoji['mean'][i]+" ", text)
      # Enlever les :
      text = re.sub(r':', '', text)

    # print('Text(convert emoji into text) : ', text)
  except:
    print("can't do it on "+text)
  
  return text

In [32]:
# nettoyer les emojis
# df_tweets_clean['emoji'] = df_tweets_clean['nettoyer ponctuaction'].apply(lambda x: convert_emojis_to_text(x))
# df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x: convert_emojis_to_text(x))
df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].progress_apply(lambda x: convert_emojis_to_text(x))

  0%|          | 0/760256 [00:00<?, ?it/s]

can't do it on RT @fadouce #Thales et Senetas  lancent une solution de chiffrement de réseau résistante au #quantique   desident_emot_keycap_1 ere  mondiale!
*️⃣Elle sera en…


In [33]:
df_tweets_clean

,liability,liability_label,user_screen_name,retweet_count,favorite_count,count_followers,tweet,cleaned
0,1,site plutôt douteux,france_soir,3,11,126112,"🎙️ ""Un Corse indépendantiste, c’est à la base ...","desident_emot_studio_microphone ️ ""Un Corse i..."
1,1,site plutôt douteux,france_soir,8,31,126112,🍄 La start-up Mycophyto propose d’enrichir le...,desident_emot_mushroom La start-up Mycophyt...
2,1,site plutôt douteux,france_soir,54,85,126112,📽 Dans un debriefing exclusif conduit avec Ari...,desident_emot_film_projector Dans un debrief...
3,1,site plutôt douteux,france_soir,21,36,126112,⚠️ Le conflit #UkraineRussie créant à la fois ...,desident_emot_warning ️ Le conflit #UkraineRu...
4,1,site plutôt douteux,france_soir,213,350,126112,"🖍️ Le dessin du jour sur #McKinseyGate, par Z_...",desident_emot_crayon ️ Le dessin du jour sur ...
...,...,...,...,...,...,...,...,...
760251,0,site plutôt fiable,wikiHow,0,2,73816,"Remember, remember, the 5th of November. https...","Remember, remember, the 5th of November. https..."
760252,0,site plutôt fiable,wikiHow,1,6,73816,#wikitips https://t.co/dzc5wobrmx https://t.co...,#wikitips https://t.co/dzc5wobrmx https://t.co...
760253,0,site plutôt fiable,wikiHow,0,6,73816,#wikitips https://t.co/g7Lg8EjTlm https://t.co...,#wikitips https://t.co/g7Lg8EjTlm https://t.co...
760254,0,site plutôt fiable,wikiHow,5,12,73816,"Thanks for this, @MichaelDapaah https://t.co/M...","Thanks for this, @MichaelDapaah https://t.co/M..."


In [34]:
# Mettre les mots en minuscule
# df_tweets_clean['minuscule'] = df_tweets_clean['tweet'].apply(lambda x : x.lower()) 
# df_tweets_clean['cleaned'] = df_tweets_clean['tweet'].apply(lambda x : x.lower()) 
df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x : x.lower()) 

In [35]:
# transformation des liens internet en "desident_link"
# df_tweets_clean['liens'] = df_tweets_clean['nettoyer @'].apply(lambda x : re.sub(r"https://t.co/[a-zA-Z\u00c0-\u00FF0-9]+", "desident_link", x))
# df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x : re.sub(r"https://[a-zA-Z\u00c0-\u00FF0-9]+", "desident_link", x))

df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x : re.sub(r"http\S+", "desident_link", x))

In [36]:
# nettoyer hashtag #
# df_tweets_clean['nettoyer #'] = df_tweets_clean['minuscule'].apply(lambda x : re.sub(r"#\w+", "", x))
df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x : re.sub(r"#\S+", " ", x))

In [37]:
# nettoyer @
# df_tweets_clean['nettoyer @'] = df_tweets_clean['nettoyer #'].apply(lambda x : re.sub(r"@\w+", "", x))
df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x : re.sub(r"@\S+", " ", x))

In [38]:
# fonction nettoyer ponctuation
import string
def Remove_Punctuation(text):
    result = "".join([ch for ch in text if ch not in string.punctuation.replace('_','')])
    return result

In [39]:
# nettoyer les ponctuation
# df_tweets_clean['nettoyer ponctuaction'] = df_tweets_clean['nettoyer ?'].apply(lambda x: Remove_Punctuation(x))
df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x: Remove_Punctuation(x))

In [40]:
# transformation des ! en "desident_exclamation"
# df_tweets_clean['nettoyer !'] = df_tweets_clean['liens'].apply(lambda x : re.sub(r"!", "desident_exclamation", x))
df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x : re.sub(r"!", "desident_exclamation", x))

In [41]:
# transformation des ? en "desident_exclamation"
# df_tweets_clean['nettoyer ?'] = df_tweets_clean['nettoyer !'].apply(lambda x : re.sub(r"[?]", "desident_question", x))
df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x : re.sub(r"[?]", "desident_question", x))

Suppression des dates

In [42]:
patterns_date = [r'\blundi\b', r'\bmardi\b', r'\bmercredi\b', r'\bjeudi\b', r'\bvendredi\b', r'\bsamedi\b', r'\bdimanche\b',
                 r'\bjanvier\b', r'\bfévrier\b', r'\bfevrier\b', r'\bmars\b', r'\bavril\b', r'\bmai\b', r'\bjuin\b', r'\bjuillet\b', r'\baoût\b', r'\baout\b', r'\bseptembre\b', r'\boctobre\b', r'\bnovembre\b', r'\bdécembre\b', r'\bdecembre\b',
                 r'\bmatin\b', r'\bmidi\b', r'\bsoir\b', r'\bnuit\b', r'\bdemain\b', r'\bhier\b', r"\baujourd'hui\b", r'\b202[0-9]\b']
# print(patterns_date)

def Replace_date(tweet):
  for pattern in patterns_date:
    tweet = re.sub(pattern, ' desident_date ', tweet)
  return tweet

In [43]:
Replace_date("test demain, mai, 2023, test")

'test  desident_date ,  desident_date ,  desident_date , test'

In [44]:
df_tweets_clean["cleaned"] = df_tweets_clean["cleaned"].apply(lambda tweet : Replace_date(tweet))

In [45]:
df_tweets_clean.head(100)

,liability,liability_label,user_screen_name,retweet_count,favorite_count,count_followers,tweet,cleaned
0,1,site plutôt douteux,france_soir,3,11,126112,"🎙️ ""Un Corse indépendantiste, c’est à la base ...",desident_emot_studio_microphone ️ un corse in...
1,1,site plutôt douteux,france_soir,8,31,126112,🍄 La start-up Mycophyto propose d’enrichir le...,desident_emot_mushroom la startup mycophyto...
2,1,site plutôt douteux,france_soir,54,85,126112,📽 Dans un debriefing exclusif conduit avec Ari...,desident_emot_film_projector dans un debrief...
3,1,site plutôt douteux,france_soir,21,36,126112,⚠️ Le conflit #UkraineRussie créant à la fois ...,desident_emot_warning ️ le conflit créant à...
4,1,site plutôt douteux,france_soir,213,350,126112,"🖍️ Le dessin du jour sur #McKinseyGate, par Z_...",desident_emot_crayon ️ le dessin du jour sur ...
...,...,...,...,...,...,...,...,...
95,1,site plutôt douteux,france_soir,19,60,126112,#CetaitDansFranceSoir : 21 septembre 2020\nhtt...,21 desident_date desident_date \ndesiden...
96,1,site plutôt douteux,france_soir,56,136,126112,#CetaitDansFranceSoir : 21 août 2020\nhttps://...,21 desident_date desident_date \ndesiden...
97,1,site plutôt douteux,france_soir,407,571,126112,"""L'émergence d'une pandémie mentale chez les p...",lémergence dune pandémie mentale chez les plus...
98,1,site plutôt douteux,france_soir,14,28,126112,"🪶 ""Disciple-renégat"" d'Alain Badiou, avec qui ...",desident_emot_feather disciplerenégat dalain...


In [46]:
# df_tweets_clean.loc[df_tweets_clean["cleaned"].str.contains("lenouveleco"), ["cleaned","tweet"]]

In [47]:
# transformation des valeurs numériques en "desident_number"
df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x : re.sub(r"\b[0-9]+", " desident_number ", x))

In [48]:
# transformation des éléments de la forme d'une heure précise en "desident_date" (pour que les heures soient vu comme des indices de date)
df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x : re.sub(r"[0-9]h[0-9][0-9]", "desident_date", x))
df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x : re.sub(r"[0-9][0-9]h[0-9][0-9]", "desident_date", x))
df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x : re.sub(r"[0-9][0-9]h", "desident_date", x))
df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x : re.sub(r"[0-9]h", "desident_date", x))

In [49]:
# transformation des éléments de la forme "desident_date" "desident_number" et "desident_number" "desident_date" en "desident_date"  (pour les formes comme "lundi 10" ou 25 décembre)
df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x : re.sub(r"desident_date desident_number", "desident_date", x))
df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x : re.sub(r"desident_number desident_date", "desident_date", x))

In [50]:
# transformation des suites de "desident_date" en un seul "desident_date"
df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x : re.sub("(desident_date)+", "desident_date ", x))

In [51]:
df_tweets_clean.head(100)

,liability,liability_label,user_screen_name,retweet_count,favorite_count,count_followers,tweet,cleaned
0,1,site plutôt douteux,france_soir,3,11,126112,"🎙️ ""Un Corse indépendantiste, c’est à la base ...",desident_emot_studio_microphone ️ un corse in...
1,1,site plutôt douteux,france_soir,8,31,126112,🍄 La start-up Mycophyto propose d’enrichir le...,desident_emot_mushroom la startup mycophyto...
2,1,site plutôt douteux,france_soir,54,85,126112,📽 Dans un debriefing exclusif conduit avec Ari...,desident_emot_film_projector dans un debrief...
3,1,site plutôt douteux,france_soir,21,36,126112,⚠️ Le conflit #UkraineRussie créant à la fois ...,desident_emot_warning ️ le conflit créant à...
4,1,site plutôt douteux,france_soir,213,350,126112,"🖍️ Le dessin du jour sur #McKinseyGate, par Z_...",desident_emot_crayon ️ le dessin du jour sur ...
...,...,...,...,...,...,...,...,...
95,1,site plutôt douteux,france_soir,19,60,126112,#CetaitDansFranceSoir : 21 septembre 2020\nhtt...,desident_number desident_date desiden...
96,1,site plutôt douteux,france_soir,56,136,126112,#CetaitDansFranceSoir : 21 août 2020\nhttps://...,desident_number desident_date desiden...
97,1,site plutôt douteux,france_soir,407,571,126112,"""L'émergence d'une pandémie mentale chez les p...",lémergence dune pandémie mentale chez les plus...
98,1,site plutôt douteux,france_soir,14,28,126112,"🪶 ""Disciple-renégat"" d'Alain Badiou, avec qui ...",desident_emot_feather disciplerenégat dalain...


In [52]:
spacy_stopwords_french = spacy.lang.fr.stop_words.STOP_WORDS  


# fonction Remove_Stopwords
def Remove_Stopwords(content_list):
  result = [word for word in content_list if word not in spacy_stopwords_french]
  return result

In [53]:
df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x : x.split())

In [54]:
# Suppression des stop words
df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x: Remove_Stopwords(x))

In [55]:
df_tweets_clean

,liability,liability_label,user_screen_name,retweet_count,favorite_count,count_followers,tweet,cleaned
0,1,site plutôt douteux,france_soir,3,11,126112,"🎙️ ""Un Corse indépendantiste, c’est à la base ...","[desident_emot_studio_microphone, ️, corse, in..."
1,1,site plutôt douteux,france_soir,8,31,126112,🍄 La start-up Mycophyto propose d’enrichir le...,"[desident_emot_mushroom, startup, mycophyto, p..."
2,1,site plutôt douteux,france_soir,54,85,126112,📽 Dans un debriefing exclusif conduit avec Ari...,"[desident_emot_film_projector, debriefing, exc..."
3,1,site plutôt douteux,france_soir,21,36,126112,⚠️ Le conflit #UkraineRussie créant à la fois ...,"[desident_emot_warning, ️, conflit, créant, fo..."
4,1,site plutôt douteux,france_soir,213,350,126112,"🖍️ Le dessin du jour sur #McKinseyGate, par Z_...","[desident_emot_crayon, ️, dessin, jour, z_cari..."
...,...,...,...,...,...,...,...,...
760251,0,site plutôt fiable,wikiHow,0,2,73816,"Remember, remember, the 5th of November. https...","[remember, remember, the, desident_number, th,..."
760252,0,site plutôt fiable,wikiHow,1,6,73816,#wikitips https://t.co/dzc5wobrmx https://t.co...,"[desident_link, desident_link]"
760253,0,site plutôt fiable,wikiHow,0,6,73816,#wikitips https://t.co/g7Lg8EjTlm https://t.co...,"[desident_link, desident_link]"
760254,0,site plutôt fiable,wikiHow,5,12,73816,"Thanks for this, @MichaelDapaah https://t.co/M...","[thanks, for, this, desident_link]"


In [56]:
# transmettre les liste de mots en phrase(texte)
def return_text(result):
  result_txt = " ".join(result)
  return result_txt

In [57]:
df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x: return_text(x))

In [58]:
df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x : re.sub(r"\b(gt)+", "", x))
df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x : re.sub(r"\b[a-zA-Z\u00c0-\u00FF0-9]\b", "", x))

In [59]:
df_tweets_clean.sample(n=10, replace=False, random_state=seed)

,liability,liability_label,user_screen_name,retweet_count,favorite_count,count_followers,tweet,cleaned
208068,0,site plutôt fiable,TVMAG,2,4,70019,"""Mon père était dans la résistance, donc le ch...",père résistance choix simple résistance dautre...
395551,0,site plutôt fiable,LT_LaTribune,0,0,7896,Secteur public: les trois centrales tentent de...,secteur public centrales tentent reconstituer ...
704718,1,site plutôt douteux,Panamza,1,0,21374,"RT @FernandoMadridd: Destellos, explosiones. ¿...",rt destellos explosiones ¿puede ser por transf...
262510,0,site plutôt fiable,presseocean,2,1,73256,#Nantes. Le grand orgue Debierre revient à Not...,grand orgue debierre revient notredamedebonpor...
459632,0,site plutôt fiable,HoustonChron,0,0,661630,Review: Houston Symphony’s Andres Fest kicks o...,review houston symphony’ andres fest kicks off...
446910,0,site plutôt fiable,lecho,0,0,64482,Comment l'Europe veut réduire sa dépendance au...,leurope veut réduire dépendance gaz russe tier...
1442,1,site plutôt douteux,france_soir,638,1339,126112,🚨 Les mises en demeures visant les personnels ...,desident_emot_police_car_light mises demeures ...
527872,0,site plutôt fiable,canalplus,27,0,3309487,"RT @CanalFootClub: La Ligue des Champions, c'e...",rt ligue champions cest desident_emot_smiling_...
246600,0,site plutôt fiable,RTLFrance,9,11,1063808,Une voiture a foncé sur une foule se rassembla...,voiture foncé foule rassemblant participer car...
45201,0,site plutôt fiable,actu17,9,19,70432,🇫🇷 Procès Maëlys : Nordahl Lelandais reconnait...,desident_emot_france procès maëlys nordahl lel...


Indique l'utilité d'un mot dans un tweet (le tagging)

In [60]:
new_column = []
i=0
for msg in df_tweets_clean['tweet']:
  if (i%1000==0):
    print('msg '+str(i))
  doc = nlp(msg)
  if (i%1000==0):
    print(doc)
  new_msg = ''
  for token in doc:
    new_msg = new_msg+token.text+'_'+token.pos_+' '
  if (i%1000==0):
    print(new_msg)
  new_column.append(new_msg)
  if (i%1000==0):
    print('\n\n')
  i = i+1

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
_SPACE Tour_PROPN d’_ADP horizon_NOUN de_ADP ce_PRON qui_PRON …_ADV 



msg 29000
Dans les années 1940, le #charbon est la principale #énergie en #Suisse. Dans les cantons de #Vaud, #Zurich et… https://t.co/BpD1R96OU3
Dans_ADP les_DET années_NOUN 1940_NUM ,_PUNCT le_DET #_ADJ charbon_NOUN est_AUX la_DET principale_NOUN #_ADP énergie_NOUN en_ADP #_NOUN Suisse_PROPN ._PUNCT Dans_ADP les_DET cantons_NOUN de_ADP #_NOUN Vaud_PROPN ,_PUNCT #_PROPN Zurich_PROPN et_CCONJ …_ADV https://t.co/BpD1R96OU3_PROPN 



msg 30000
#Lelion avec #DanyBoon
Dispo en replay jusqu'au 4/3/22 sur #Auvio https://t.co/Q50yPrzk8f
#_NOUN Lelion_NOUN avec_ADP #_NOUN DanyBoon_PROPN 
_SPACE Dispo_NOUN en_ADP replay_NOUN jusqu'_ADP au_DET 4/3/22_PROPN sur_ADP #_NOUN Auvio_PROPN https://t.co/Q50yPrzk8f_PROPN 



msg 31000
Les années ont passé, Marine vient d’avoir 50 ans et joue dans “Alice Nevers” depuis 18 ans 🔥 https://t.co/l7n8FaUhUo
Les_DE

In [61]:
df_tweets_clean['tagged'] = new_column
df_tweets_clean

,liability,liability_label,user_screen_name,retweet_count,favorite_count,count_followers,tweet,cleaned,tagged
0,1,site plutôt douteux,france_soir,3,11,126112,"🎙️ ""Un Corse indépendantiste, c’est à la base ...",desident_emot_studio_microphone ️ corse indépe...,"🎙_NOUN ️_ADJ ""_PUNCT Un_DET Corse_PROPN indépe..."
1,1,site plutôt douteux,france_soir,8,31,126112,🍄 La start-up Mycophyto propose d’enrichir le...,desident_emot_mushroom startup mycophyto propo...,🍄_PROPN _SPACE La_DET start_NOUN -_PUNCT up_X...
2,1,site plutôt douteux,france_soir,54,85,126112,📽 Dans un debriefing exclusif conduit avec Ari...,desident_emot_film_projector debriefing exclus...,📽_PROPN Dans_ADP un_DET debriefing_NOUN exclus...
3,1,site plutôt douteux,france_soir,21,36,126112,⚠️ Le conflit #UkraineRussie créant à la fois ...,desident_emot_warning ️ conflit créant fois pa...,⚠_PROPN ️_VERB Le_DET conflit_NOUN #_ADJ Ukrai...
4,1,site plutôt douteux,france_soir,213,350,126112,"🖍️ Le dessin du jour sur #McKinseyGate, par Z_...",desident_emot_crayon ️ dessin jour z_caricatur...,🖍_NOUN ️_ADV Le_DET dessin_NOUN du_DET jour_NO...
...,...,...,...,...,...,...,...,...,...
760251,0,site plutôt fiable,wikiHow,0,2,73816,"Remember, remember, the 5th of November. https...",remember remember the desident_number th of no...,"Remember_PROPN ,_PUNCT remember_PROPN ,_PUNCT ..."
760252,0,site plutôt fiable,wikiHow,1,6,73816,#wikitips https://t.co/dzc5wobrmx https://t.co...,desident_link desident_link,#_NOUN wikitips_PROPN https://t.co/dzc5wobrmx_...
760253,0,site plutôt fiable,wikiHow,0,6,73816,#wikitips https://t.co/g7Lg8EjTlm https://t.co...,desident_link desident_link,#_NOUN wikitips_PROPN https://t.co/g7Lg8EjTlm_...
760254,0,site plutôt fiable,wikiHow,5,12,73816,"Thanks for this, @MichaelDapaah https://t.co/M...",thanks for this desident_link,"Thanks_X for_X this_X ,_PUNCT @MichaelDapaah_P..."


In [62]:
df_tweets_clean.to_csv('Tweets_nettoyés_050422_avecemoji.csv')
!cp Tweets_nettoyés_050422_avecemoji.csv "/content/drive/My Drive/PFE"

Pour l'enregistrement en format pickle

In [63]:
import pickle

In [64]:
df_tweets_clean.to_pickle(DATA_PATH+"Tweets_nettoyés_050422_avecemoji.pkl")